In [1]:
import pandas as pd
import numpy as np
import time
import math

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
df_acc = pd.read_csv('acc_highway_name.csv',encoding='utf-8')
df_acc = df_acc.drop('Unnamed: 0',1)
print(len(df_acc))
df_acc.head(1)


26486


,발생시간,메세지,경도,위도,고속도로,콘존ID
0,2014-12-01 06:49:02,경인고속도로 부천-신월 3차로 추돌사고 지체 -,126.826106822352,37.5225191751392,경인고속도로,NaN


In [3]:
#df_acc.to_csv('acc_highway_name.csv')

In [4]:
# 콘존ID - 콘존명 데이터
df_cz_name = pd.read_csv('czid_to_gugan.txt',sep="\t",encoding='CP949')

cols = df_cz_name.columns
cols = cols[0].split(',')

df_cz_name.index = range(len(df_cz_name))
df_cz_name.columns=range(1)

df_cz_name = df_cz_name[0].str.split(',',n=12,expand=True)
df_cz_name.columns = cols
print(len(df_cz_name))
df_cz_name = df_cz_name.drop(['시작노드ID','종료노드ID','차로수','제한속도','버스전용차로유무','도로등급구분코드'],1)
df_cz_name.head(1)

1438


,콘존ID,콘존길이,기점종점방향구분코드,노선번호,노선구성순번,콘존명
0,0010CZE005,200.00,E,0010,1,경부고속국도시점-구서IC


In [5]:
# 콘존ID - 좌표 데이터 위도,경도 변환

df_cz_xy = pd.read_csv('acc_conzone.csv',sep=',',encoding='CP949')
df_cz_xy = df_cz_xy.drop('Unnamed: 0',1)
str_arr= ['콘존ID','방향','위도','경도','노선명']
df_cz_xy = df_cz_xy[str_arr]
print(len(df_cz_xy))
df_cz_xy.head(1)

6301


,콘존ID,방향,위도,경도,노선명
0,0010CZE010,E,35.256977,129.101614,경부선


In [6]:
# 고속도로 설치 정보 데이터
df_ic = pd.read_csv('highway_sisul.csv',sep=',',encoding="CP949")
df_ic = df_ic.set_index('Unnamed: 0')
print(len(df_ic))
df_ic.head(2)
#df_ic[df_ic['노선명'].str.contains('남해선')].head(1)

407


,IC/JCT코드,IC/JCT명,노선명,노선코드,X좌표값,Y좌표값
Unnamed: 0,,,,,,
0,0100I00013,진주IC,남해선(순천-부산),100,128.118649,35.162711
1,0100I00015,진성IC,남해선(순천-부산),100,128.225744,35.191538


In [7]:
for s in ['경부','중부','중부내륙','경인','제2경인','영동']:
    print(df_ic[df_ic['노선명']=='%s선'%s].head(1).values[0][2:4])

['경부선' 10]
['중부선' 352]
['중부내륙선' 450]
['경인선' 1200]
['제2경인선' 1100]
['영동선' 500]


In [8]:
#고속도로별 노선코드 빼기

tmp = list(df_acc['고속도로'])
#print(len(tmp))
tmp = set(tmp)

a = pd.DataFrame()

for s in tmp:
    s=s.split('고속')[0]
    
    a=a.append(df_ic[df_ic['노선명']=='%s선'%s].head(1))
a=a.append(df_ic[df_ic['노선명'].str.contains('남해선')].head(1))
tmps = []
for tmp in a['노선코드']:
    tmp = str('%04d'%int(tmp))
    tmps.append(tmp)
a['노선코드'] = tmps
a = pd.merge(a['노선명'],a['노선코드'],on=a.index)
a= a.drop('key_0',1)
a

,노선명,노선코드
0,중부내륙선,0450
1,경부선,0010
2,서해안선,0150
3,영동선,0500
4,중부선,0352
5,서울외곽순환선,1000
6,호남선,0251
7,경인선,1200
8,남해선(순천-부산),0100


In [18]:
n_name = list(a['노선명'])
n_code = list(a['노선코드'])

ic_dic = {}

for i in range(len(n_name)):

    tt = []
    tmp = list(df_cz_name['콘존명'][df_cz_name['노선번호']==n_code[i]])
    t=''
    for t in tmp:
        tt.append(t.split('-')[0])
    tt.append(t.split('-')[1])
    #tt=list(set(tt))
    #tt=set(tt) 이거쓰면안댐
    tmp = n_name[i]
    
    if ('남해') in tmp:
        tmp='남해선'
    ic_dic[tmp[:-1]+'고속도로']=tt

ic_dic
#이거 순서 맞는지 확인 필요


{'중부내륙고속도로': ['내서JC',
  '칠원TG',
  '칠원JC',
  '칠서IC',
  '남지IC',
  '영산IC',
  '창녕IC',
  '현풍JC',
  '고령JC',
  '남성주IC',
  '성주IC',
  '남김천IC',
  '김천JC',
  '선산IC',
  '낙동JC',
  '상주IC',
  '북상주IC',
  '점촌함창IC',
  '문경새재IC',
  '연풍IC',
  '괴산IC',
  '충주IC',
  '북충주IC',
  '충주JC',
  '감곡IC',
  '여주JC',
  '남여주IC',
  '서여주IC',
  '북여주IC',
  '남양평Hi',
  '칠원TG',
  '칠원JC',
  '칠서IC',
  '남지IC',
  '영산IC',
  '창녕IC',
  '현풍JC',
  '고령JC',
  '남성주IC',
  '성주IC',
  '남김천IC',
  '김천JC',
  '선산IC',
  '낙동JC',
  '상주IC',
  '북상주IC',
  '점촌함창IC',
  '문경새재IC',
  '연풍IC',
  '괴산IC',
  '충주IC',
  '북충주IC',
  '충주JC',
  '감곡IC',
  '여주JC',
  '남여주IC',
  '서여주IC',
  '북여주IC',
  '남양평Hi',
  '양평IC',
  '남양평Hi'],
 '경부고속도로': ['경부고속국도시점',
  '구서IC',
  '영락IC',
  '부산TG',
  '노포IC',
  '노포JC',
  '양산JC',
  '양산IC',
  '통도사Hi',
  '통도사IC',
  '서울산IC',
  '언양JC',
  '활천IC',
  '경주IC',
  '건천IC',
  '영천JC',
  '영천IC',
  '경산IC',
  '동대구JC',
  '도동JC',
  '북대구IC',
  '금호JC',
  '칠곡물류IC',
  '왜관IC',
  '남구미IC',
  '구미IC',
  '김천JC',
  '동김천IC',
  '김천IC',
  '추풍령IC',
  '황간IC',
  '영동IC',
  '금강IC'

{'경부고속도로',
 '경인고속도로',
 '남해고속도로',
 '대전당진고속도로',
 '서울외곽순환고속도로',
 '서해안고속도로',
 '영동고속도로',
 '중부고속도로',
 '중부내륙고속도로',
 '호남고속도로'}

In [82]:

df_cz_tmp['시작경도'] = 0.0
df_cz_tmp['시작위도'] = 0.0
df_cz_tmp['끝경도']=0
df_cz_tmp['끝위도']=0

count=0
df_ic['IC/JCT명'] = df_ic['IC/JCT명'].str.upper()
for ic in df_cz_tmp['시작']:
    #m_find = False
    for dfic in df_ic['IC/JCT명']:
        
        if ic in dfic:   
            m_find=True
            xx=df_ic['X좌표값'][df_ic['IC/JCT명']==dfic].copy()
            yy=df_ic['Y좌표값'][df_ic['IC/JCT명']==dfic].copy()

            try:
                xx=float(xx)
                yy=float(yy)
            except:
                xx=xx[xx.index[0]].copy()
                yy=yy[yy.index[0]].copy()
                xx = float(xx)
                yy = float(yy)
            
            df_cz_tmp['시작위도'][df_cz_tmp['시작']==ic] = yy
            df_cz_tmp['시작경도'][df_cz_tmp['시작']==ic] = xx
            count+=1

count=0
df_ic['IC/JCT명'] = df_ic['IC/JCT명'].str.upper()
for ic in df_cz_tmp['끝']:
    #m_find = False
    for dfic in df_ic['IC/JCT명']:
        
        if ic in dfic:   
            m_find=True
            xx=df_ic['X좌표값'][df_ic['IC/JCT명']==dfic].copy()
            yy=df_ic['Y좌표값'][df_ic['IC/JCT명']==dfic].copy()

            try:
                xx=float(xx)
                yy=float(yy)
            except:
                xx=xx.loc[xx.index[0]]
                yy=yy.loc[yy.index[0]]
                xx = float(xx)
                yy = float(yy)
            
            df_cz_tmp['끝위도'][df_cz_tmp['끝']==ic] = yy
            df_cz_tmp['끝경도'][df_cz_tmp['끝']==ic] = xx
            count+=1

NameError: name 'df_cz_tmp' is not defined

70
67


,콘존ID,콘존길이,기점종점방향구분코드,노선번호,노선구성순번,콘존명,시작,끝,시작경도,시작위도,끝경도,끝위도
843,0500CZE005,560.00,E,0500,1,영동고속국도시점-서창JC,영동고속국도시점,서창JC,0.000000,0.000000,126.739118,37.432337
844,0500CZE010,4440.00,E,0500,2,서창JC-월곶JC,서창JC,월곶JC,126.739118,37.432337,0.000000,0.000000
845,0500CZE020,2960.00,E,0500,3,월곶JC-군자JC,월곶JC,군자JC,0.000000,0.000000,0.000000,0.000000
846,0500CZE025,2080.00,E,0500,4,군자JC-군자TG,군자JC,군자TG,0.000000,0.000000,0.000000,0.000000
847,0500CZE030,2500.00,E,0500,5,군자TG-서안산IC,군자TG,서안산IC,0.000000,0.000000,126.800796,37.349730
848,0500CZE040,3330.00,E,0500,6,서안산IC-안산IC,서안산IC,안산IC,126.800796,37.349730,126.849215,37.347671
849,0500CZE050,1230.00,E,0500,7,안산IC-안산JC,안산IC,안산JC,126.849215,37.347671,126.858377,37.345344
850,0500CZE060,5600.00,E,0500,8,안산JC-둔대JC,안산JC,둔대JC,126.858377,37.345344,126.905945,37.323563
851,0500CZE065,1720.00,E,0500,9,둔대JC-군포IC,둔대JC,군포IC,126.905945,37.323563,126.935591,37.329003
852,0500CZE070,1390.00,E,0500,10,군포IC-동군포IC,군포IC,동군포IC,126.935591,37.329003,126.935591,37.329003


In [13]:
#x = 127
#y = 37.5
#x,y = 126.852964704579, 37.3433700997067
## x,y 에서 난 사고에 콘존ID를 붙힐거임
#print(len(df_cz_tmp))


def FindIC(x,y):

    df_x1 = df_cz_tmp['시작경도']
    df_y1 = df_cz_tmp['시작위도']
    d=0
    c=0
    m=0
    mx=0
    for x1 in df_cz_tmp['시작경도']:
        if x1 ==0.0:
            continue
        y1 = df_cz_tmp['시작위도'][df_cz_tmp['시작경도']==x1]
        y1 = y1.values[0]
        a = (x-x1)*(x-x1)
        b = (y-y1)*(y-y1)
        c = a+b
        c = math.sqrt(c)
        if c < d:
            m = c
            mx = x1
        d = c
        #print('%.4f,%.4f'%(x1,y1))
    
    return df_cz_tmp['시작'][df_cz_tmp['시작경도']==mx].values[0]
    
    
#print('거리 ', m)
#print('가장짧은거리의 IC')
#df_cz_tmp['시작'][df_cz_tmp['시작경도']==mx]

In [16]:
print(FindIC(126.852964704579, 37.3433700997067))
print(FindIC(126.3, 37.3433700997067))
print(FindIC(126.1, 37.3433700997067))
FindIC(126.776906,37.358812)

안산IC
서창JC
서창JC


'서안산IC'

In [18]:
df_cz_tmp.head(5)

,콘존ID,콘존길이,기점종점방향구분코드,노선번호,노선구성순번,콘존명,시작,끝,시작경도,시작위도,끝경도,끝위도
843,0500CZE005,560.00,E,0500,1,영동고속국도시점-서창JC,영동고속국도시점,서창JC,0.000000,0.000000,126.739118,37.432337
844,0500CZE010,4440.00,E,0500,2,서창JC-월곶JC,서창JC,월곶JC,126.739118,37.432337,0.000000,0.000000
845,0500CZE020,2960.00,E,0500,3,월곶JC-군자JC,월곶JC,군자JC,0.000000,0.000000,0.000000,0.000000
846,0500CZE025,2080.00,E,0500,4,군자JC-군자TG,군자JC,군자TG,0.000000,0.000000,0.000000,0.000000
847,0500CZE030,2500.00,E,0500,5,군자TG-서안산IC,군자TG,서안산IC,0.000000,0.000000,126.800796,37.349730


In [ ]:
df_

In [19]:
df_ic['X좌표값'][df_ic['IC/JCT명']=='대전IC']

Unnamed: 0
86    127.450756
Name: X좌표값, dtype: float64

In [40]:
float(df_ic.loc[:]['X좌표값'][df_ic['IC/JCT명']==dfic])

126.73911799999999

126.90594499999999